In [2]:
!cd paylesshealth/ && dolt sql -q "select * from hospitals where cdm_indirect_url is not null;" -r csv > all_nulls.csv

In [3]:
import pandas as pd
# df = pd.read_csv('paylesshealth/hospitals.csv')
df = pd.read_csv('./paylesshealth-1/all_nulls.csv')

df2 = pd.read_csv('finished_possibles.csv')
ccn_list = df2["ccn"].tolist()

df = df[~df['ccn'].isin(ccn_list)]
# df = df.head(100)

In [15]:
import aiohttp
import asyncio
import re
import cgi
from pathlib import Path
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import traceback as tb

In [5]:
def create_headers(url):
    headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Host": urlparse(url).netloc,
    "Referer": urlparse(url).netloc
    }
    return headers

In [18]:
async def checker(session, url):
    find_strings = ["price transparency", "standard charges", "pricing", "billing", "prices", "price listing", "charges", "transparency", "standard services"]
    url_string = ["price-transparency", "standard-charges", "pricing", "billing", "prices", "price-list", "price-listing", "charges", "transparency", "standard-services"]
    if pd.isna(url):
        return False
    
    # if "/../" in url or "|" in url: # aiohttp is kinda not smart
    #     return False

    headers = create_headers(url)
    
    try:
        async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
            if r:
                r = await r.text()
                soup = BeautifulSoup(r, "html.parser")
                
                r = r.lower()
                # Check for common price transparency names
                if any(string in r for string in find_strings):
                    matching_str = [string for string in find_strings if string in r]

                    possible_indirects = []
                    for t_url in soup.find_all('a'):
                        t_url = t_url.get("href")
                        
                        if t_url:
                            if any(string in t_url for string in url_string):
                                if t_url not in possible_indirects:
                                    possible_indirects.append(t_url)

                    if not possible_indirects:
                        # possible_indirects = pd.NA
                        return pd.NA, pd.NA
                    else:
                        possible_indirects = " | ".join(possible_indirects)
                    
                    return possible_indirects, matching_str
                
                else:
                    return pd.NA, pd.NA
                    
    except aiohttp.ClientResponseError as e:
#         print("ERROR", e, url)
        return pd.NA, pd.NA

    except asyncio.TimeoutError:
        return pd.NA, pd.NA

    except aiohttp.ClientConnectionError:
        return pd.NA, pd.NA    
    
    except KeyboardInterrupt:
        import os; os.exit()

    except UnicodeDecodeError:
        tb.print_exc()        
    except Exception as e:
        tb.print_exc()
        print(e)
        return pd.NA, pd.NA

In [17]:
from tqdm.asyncio import tqdm

async def main():
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df[["possible_indirect", "found_string"]] = await tqdm.gather(*(checker(session, url) for url in df["homepage"]), leave=True, position=0)
        return df

In [19]:
t = await main()

 33%|███▎      | 460/1389 [00:15<01:03, 14.55it/s] Traceback (most recent call last):
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_22208\3244036188.py", line 15, in checker
    r = await r.text()
  File "c:\Users\adria\Miniconda3\envs\scraper\lib\site-packages\aiohttp\client_reqrep.py", line 1085, in text
    return self._body.decode(  # type: ignore[no-any-return,union-attr]
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 79817: invalid start byte
 33%|███▎      | 462/1389 [00:15<01:12, 12.75it/s]

'utf-8' codec can't decode byte 0x97 in position 79817: invalid start byte


100%|██████████| 1389/1389 [01:30<00:00, 15.28it/s]


ValueError: Columns must be same length as key

In [9]:
t

NameError: name 't' is not defined

In [84]:
t.dropna(subset=["found_string"], inplace=True)
t

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url,possible_indirect,found_string
177,173027,"WESLEY REHABILITATION HOSPITAL, AN AFFILIATE O...",KS,WICHITA,https://encompasshealth.com/,NaN,NaN,/inpatient-rehabilitation/resources-and-educat...,[transparency]
178,673032,RELIANT REHABILITATION HOSPITAL OF CENTRAL TEXAS,TX,ROUND ROCK,https://encompasshealth.com/,NaN,NaN,/inpatient-rehabilitation/resources-and-educat...,[transparency]
179,670172,SIGNATURE CARE EMERGENCY CENTER-MISSION BEND,TX,HOUSTON,https://ercare24.com/,NaN,NaN,https://ercare24.com/payment-and-billing/,[billing]
180,670173,SIGNATURE CARE EMERGENCY CENTER-MONTROSE,TX,HOUSTON,https://ercare24.com/,NaN,NaN,https://ercare24.com/payment-and-billing/,[billing]
181,670178,"TC JESTER EMERGENCY CENTER, LLC",TX,HOUSTON,https://ercare24.com/,NaN,NaN,https://ercare24.com/payment-and-billing/,[billing]
...,...,...,...,...,...,...,...,...,...
1504,340168,WILMINGTON TREATMENT CENTER,NC,WILMINGTON,https://www.wilmingtontreatment.com/,NaN,NaN,/admissions/insurance-payment-information/rate...,[pricing]
1510,521347,WESTERN WISCONSIN HEALTH,WI,BALDWIN,https://www.wwhealth.org/,NaN,NaN,https://www.wwhealth.org/your-visit/price-tran...,"[price transparency, charges, transparency]"
1511,451346,YOAKUM COMMUNITY HOSPITAL,TX,YOAKUM,https://www.yoakumhospital.org/,NaN,NaN,https://www.yoakumhospital.org/about-yoakum-ho...,"[pricing, billing, transparency]"
1513,270091,YELLOWSTONE SURGERY CENTER LLC,MT,BILLINGS,https://yellowstonesurgerycenter.com/,NaN,NaN,https://yellowstonesurgerycenter.com/about-yel...,"[pricing, billing, charges]"


In [71]:
t.to_csv("possible_indirects.csv", index=False)